In [3]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/alice/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [126]:
df = pd.read_csv("ROCStories_winter2017.csv")

In [127]:
df.head()

,storyid,storytitle,sentence1,sentence2,sentence3,sentence4,sentence5
0,8bbe6d11-1e2e-413c-bf81-eaea05f4f1bd,David Drops the Weight,David noticed he had put on a lot of weight re...,He examined his habits to try and figure out t...,He realized he'd been eating too much fast foo...,He stopped going to burger places and started ...,"After a few weeks, he started to feel much bet..."
1,0beabab2-fb49-460e-a6e6-f35a202e3348,Frustration,Tom had a very short temper.,One day a guest made him very angry.,He punched a hole in the wall of his house.,Tom's guest became afraid and left quickly.,Tom sat on his couch filled with regret about ...
2,87da1a22-df0b-410c-b186-439700b70ba6,Marcus Buys Khakis,Marcus needed clothing for a business casual e...,All of his clothes were either too formal or t...,He decided to buy a pair of khakis.,The pair he bought fit him perfectly.,Marcus was happy to have the right clothes for...
3,2d16bcd6-692a-4fc0-8e7c-4a6f81d9efa9,Different Opinions,Bobby thought Bill should buy a trailer and ha...,Bill thought a truck would be better for what ...,Bobby pointed out two vehicles were much more ...,Bill was set in his ways with conventional thi...,He ended up buying the truck he wanted despite...
4,c71bb23b-7731-4233-8298-76ba6886cee1,Overcoming shortcomings,John was a pastor with a very bad memory.,He tried to memorize his sermons many days in ...,He decided to learn to sing to overcome his ha...,He then made all his sermons into music and sa...,His congregation was delighted and so was he.


In [7]:
for i in range(20):
    print(df.sentence1[i])

David noticed he had put on a lot of weight recently.
Tom had a very short temper.
Marcus needed clothing for a business casual event.
Bobby thought Bill should buy a trailer and haul it with his car.
John was a pastor with a very bad memory.
Melody's parents surprised her with a trip to the big aquarium.
The math teacher announced a pop quiz as class began.
My first girlfriend i met on the internet.
I got Charlie Horse when I was four years old.
Laura loved corn.
Andy was invited to a Halloween party.
Luke was playing hockey at school.
Robbie was competing in a cross country meet.
Jude was very excited about his college graduation ceremony.
Beth sent a letter to Santa Claus.
Our granddaughter Anna is very fussy about her clothes.
Jake needed a ride to the store.
I'd been looking for a stand for my TV.
Sally had a root canal this morning, as she had a damaged root.
My dog is terrified of thunder.


In [8]:
for i in range(20):
    print(df.sentence2[i])

He examined his habits to try and figure out the reason.
One day a guest made him very angry.
All of his clothes were either too formal or too casual.
Bill thought a truck would be better for what he needed.
He tried to memorize his sermons many days in advance but to no avail.
Melody took a nap during the two hour car ride to the aquarium.
While some students complained, he began passing out the quiz.
She lives about 4 hours away from me.
He's a brown stuffed horse, and at 35 I still sleep with him at night.
So she decided to grow some in her backyard.
Andy figured that for dramatic effect, he should color his hair.
The game was tied and almost over.
He was halfway through when his leg cramped up.
On the way to the arena, he got stuck in traffic.
She received a letter back in the mail.
She is only two, but wants to pick her outfits.
His girlfriend was working and wouldn't take him.
I looked everywhere, but the ones I saw were expensive.
After the procedure, the dentist wrote her a pre

In [9]:
len(df)

52665

In [10]:
tokens_1 = word_tokenize(" ".join(df.sentence1))

In [12]:
len(df["sentence1"])

52665

In [18]:
def build_vocab(df):
    tokens = []
    columns = ["sentence1", "sentence2", "sentence3", "sentence4", "sentence5"]
    for col in columns:
        tok_col = word_tokenize(" ".join(df[col]))
        tokens += tok_col
    unique_tokens = list(set(tokens))
    return tokens, unique_tokens

In [19]:
tokens, unique_tokens = build_vocab(df)

In [20]:
len(unique_tokens)

32296

In [21]:
unique_tokens_1 = list(set(tokens_1))

In [22]:
len(unique_tokens_1)

16396

In [39]:
vocab = {k:v for k,v in enumerate(unique_tokens_1)}

In [24]:
import json

In [25]:
with open("vocab_ROC_sentence1.json", "w") as f:
    json.dump(vocab, f)

In [128]:
def get_sentences(df, max_samples=20000):
    sentences = df.sentence1[:max_samples]
    return sentences

In [68]:
def get_vocab(sentences):
    tokenize_func=lambda t : word_tokenize(t.lower())
    #tokens = word_tokenize(' '.join(sentences))
    tokenized_sentences = sentences.apply(tokenize_func)
    tokenized_sentences = tokenized_sentences.values
    tokens = [w for s in tokenized_sentences for w in s]
    unique_tokens = list(set(tokens))
    vocab = {v:k for k,v in enumerate(unique_tokens)}
    return tokens, vocab

In [124]:
def split_train_test(sentences, train_size=10000, val_size=5000, test_size=5000):
    train_sentences = sentences[:train_size]
    val_sentences = sentences[train_size:train_size+val_size]
    test_sentences = sentences[train_size+val_size:train_size+val_size+test_size]
    train_data = split_dataset_elements(train_sentences)
    val_data = split_dataset_elements(val_sentences)
    test_data = split_dataset_elements(test_sentences)
    return train_data, val_data, test_data

In [109]:
def tokenize(sentences, vocab):
    tokenize_func=lambda t : word_tokenize(t.lower())
    tok_to_id_func = lambda t: [vocab[w] for w in t]
    tokenized_sentences = sentences.apply(tokenize_func)
    tokens_id = tokenized_sentences.apply(tok_to_id_func)
    #TODO: split between input and target sentence. 
    df = pd.DataFrame()
    df["input_sentence"] = tokens_id.apply(lambda t: t[:-1])
    df["target_sentence"] = tokens_id.apply(lambda t: t[1:])
    len_sentences = tokens_id.apply(len)
    max_len = len_sentences.max() - 1
    df["attention_mask"] = len_sentences.apply(lambda t: [1]*(t-1) + [0]*(max_len - (t-1)))
    print("max len", max_len)
    pad_func = lambda t: t + [0]*(max_len-len(t))
    padded_token_ids = df.apply(pad_func)
    return padded_token_ids, len_sentences

In [133]:
def split_dataset_elements(dataset):
    input_sentence = dataset.input_sentence
    target_sentence = dataset.target_sentence
    attention_mask = dataset.attention_mask
    return input_sentence, target_sentence, attention_mask

In [129]:
sentences = get_sentences(df)

In [130]:
tokens, vocab = get_vocab(sentences)

In [131]:
dataset, len_sentences = tokenize(sentences, vocab)

max len 18


In [134]:
train_data, val_data, test_data = split_train_test(dataset)

In [135]:
input_train, target_train, _ = train_data

In [136]:
input_train.values

array([list([9167, 8594, 8287, 4020, 5094, 5664, 9050, 1210, 299, 7885, 37]),
       list([1793, 4020, 9050, 5612, 3883, 80]),
       list([9335, 3956, 6190, 278, 9050, 8115, 9016, 4629]), ...,
       list([7180, 5086, 9050, 775, 9294, 988, 4195, 9532, 5249, 4565]),
       list([1509, 8086, 5555, 2867, 3286, 8540, 1412, 6725, 3962, 1672]),
       list([1793, 5086, 874, 5555, 9050, 8614, 2999])], dtype=object)

In [112]:
len(tokens_id.attention_mask[0])

18

In [84]:
len(vocab)

9627

In [120]:
import numpy as np

In [122]:
data =tokens_id.values
data_b = np.reshape(data, newshape=(data.shape[0], 18, data.shape[-1]))

ValueError: cannot reshape array of size 60000 into shape (20000,18,3)

In [116]:
array = np.zeros(data.shape[0], 18, data.shape[-1])
for ind

[9167, 8594, 8287, 4020, 5094, 5664, 9050, 1210, 299, 7885, 37]